In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Libraries and Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
train_data = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data/'
test_data = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/test_data/test_data/'

train_df = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Train.csv'
test_df = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Test.csv'
sub = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/SampleSubmission.csv'

In [4]:
train = pd.read_csv(train_df)
test = pd.read_csv(test_df)

In [5]:
# Display images

# Band descriptions
band_descriptions = [
    "Red", "Green", "Blue", "Near Infrared",
    "Descending VV (Vertical-Vertical)", "Descending VH (Vertical-Horizontal)",
    "Descending Diff VV", "Descending Diff VH",
    "Ascending VV (Vertical-Vertical)", "Ascending VH (Vertical-Horizontal)",
    "Ascending Diff VV", "Ascending Diff VH"
]

# Randomly select 2 images to display

# for image_id in example_ids:
#     img_norm = load_and_normalize_images(image_id, train_data)
    
#     # Plot all 12 bands in a 3x4 grid
#     fig, axes = plt.subplots(3, 4, figsize=(20, 12))  # 3 rows, 4 columns for 12 plots
#     fig.suptitle(f"Sample Image ID: {image_id} - All 12 Bands", fontsize=16)

#     # Display each of the 12 bands with descriptions
#     for band in range(12):
#         row = band // 4  # Calculate row index (0, 1, or 2)
#         col = band % 4   # Calculate column index (0 to 3)
#         axes[row, col].imshow(img_norm[:, :, band], cmap='gray')
#         axes[row, col].set_title(f"Band {band + 1}: {band_descriptions[band]}")
#         axes[row, col].axis('off')

#     plt.subplots_adjust(wspace=0.3, hspace=0.4)  # Adjust spacing between plots
#     plt.show()

In [6]:
def load_image(image_id, path):
    image_path = os.path.join(path, f'{image_id}.npy')
    img = np.load(image_path)

    return img

def normalize_image(band):
    # Nomalize image (0-1 range)
    band_min, band_max = (band.min(), band.max())
    return ((band-band_min)/((band_max - band_min)))

In [7]:
def create_spectral_indices(image):
    # Extract individual bands
    red = image[:, :, 0]       # Band 1: Red
    green = image[:, :, 1]     # Band 2: Green
    blue = image[:, :, 2]      # Band 3: Blue
    nir = image[:, :, 3]       # Band 4: Near Infrared
    vv_desc = image[:, :, 4]   # Band 5: Descending VV
    vh_desc = image[:, :, 5]   # Band 6: Descending VH
    
    # Avoid division by zero
    epsilon = 1e-5
    
    # 1. NDVI (Normalized Difference Vegetation Index)
    ndvi = (nir - red) / (nir + red + epsilon)
    
    # 2. NDWI (Normalized Difference Water Index)
    ndwi = (green - nir) / (green + nir + epsilon)
    
    # 3. EVI (Enhanced Vegetation Index)
    # evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1 + epsilon))
    
    # 4. SAVI (Soil Adjusted Vegetation Index) - with L=0.5
    L = 0.5
    savi = ((nir - red) / (nir + red + L + epsilon)) * (1 + L)
    
    # 5. Radar Vegetation Index (using VV and VH)
    # rvi = 4 * vh_desc / (vv_desc + vh_desc + epsilon)
    
    # 6. Simple band ratio features
    # nir_red_ratio = nir / (red + epsilon)
    # vh_vv_ratio = vh_desc / (vv_desc + epsilon)
    
    # Stack the original image with the new indices
    # Reshape indices to match image dimensions for stacking
    # , nir_red_ratio, vh_vv_ratio'
    # rgb = np.stack([red, green, blue], axis=2)
    indices = np.stack([ndvi, ndwi, savi], axis=2)
    
    # Concatenate original image with new indices
    enhanced_image = np.concatenate([image, indices], axis=2)
    
    return enhanced_image


In [8]:
from skimage.feature import graycomatrix, graycoprops
from scipy import ndimage

def extract_advanced_features(img):
    """Extract topographic and texture features from image bands."""
    
    elevation_proxy = img[:, :, 3]  # Near Infrared band
    
    # 1. Calculate slope (gradient magnitude)
    dx = ndimage.sobel(elevation_proxy, axis=0)
    dy = ndimage.sobel(elevation_proxy, axis=1)
    slope = np.hypot(dx, dy)  
    
    # 2. Calculate aspect (gradient direction)
    aspect = np.arctan2(dy, dx)
    
    # 3. Calculate curvature (approximation using Laplacian)
    curvature = ndimage.gaussian_filter(elevation_proxy, sigma=2) # smoothen first
    curvature = ndimage.laplace(curvature)
    
    # 4. Calculate texture features from GLCM (on SAR bands)
    # Rescale to 0-255 and convert to uint8 for GLCM
    # radar_band = img[:, :, 0]  # Using Descending VV polarization
    # radar_band_rescaled = (radar_band * 255).astype(np.uint8)
    
    # # Compute GLCM on a downsampled version if memory is an issue
    # step_size = 2  # Skip pixels to reduce computation
    # distances = [1]  # Pixel pairs 1 pixel apart
    # angles = [0, 45, 90, 135]  # 0, 45, 90, 135 degrees
    
    # glcm = graycomatrix(radar_band_rescaled[::step_size, ::step_size], 
    #                     distances=distances, angles=angles, 
    #                     levels=256, symmetric=True, normed=True)
    
    # # Extract properties
    # contrast = graycoprops(glcm, 'contrast').mean()
    # dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
    # homogeneity = graycoprops(glcm, 'homogeneity').mean()
    # # energy = graycoprops(glcm, 'energy').mean()
    # correlation = graycoprops(glcm, 'correlation').mean()
    # mean = graycoprops(glcm, 'mean').mean()
    # variance = graycoprops(glcm, 'variance').mean()
    # entropy = graycoprops(glcm, 'entropy').mean()
    
    # # Create uniform-sized feature maps from scalar GLCM stats
    # h, w = img.shape[0], img.shape[1]
    # contrast_map = np.ones((h, w)) * contrast
    # dissimilarity_map = np.ones((h, w)) * dissimilarity
    # homogeneity_map = np.ones((h, w)) * homogeneity
    # # energy_map = np.ones((h, w)) * energy
    # correlation_map = np.ones((h, w)) * correlation
    # mean_map = np.ones((h, w)) * mean
    # variance_map = np.ones((h, w)) * variance
    # entropy_map = np.ones((h, w)) * entropy

    # print("Homogeneity range:", np.min(homogeneity_map), np.max(homogeneity_map))
    # print("Entropy range:", np.min(entropy_map), np.max(entropy_map))
    # print("Correlation range:", np.min(correlation_map), np.max(correlation_map))
    # print("Mean range:", np.min(mean_map), np.max(mean_map))
    
    
    # Stack features as additional bands
    topo_texture_features = np.stack([
        slope, aspect, curvature
        # contrast_map, dissimilarity_map, homogeneity_map, 
        # entropy_map, correlation_map, mean_map, variance_map
    ], axis=2)
    
    return topo_texture_features

In [9]:
def load_enhanced_normalized_images(image_id, path):
    raw_img = load_image(image_id, path)
    enhanced_img = create_spectral_indices(raw_img)
    enhanced_img = normalize_image(enhanced_img)
    # feature_img = extract_advanced_features(enhanced_img)

    # enhanced_img = np.concatenate([enhanced_img, feature_img], axis=2)
    
    return enhanced_img[:, :, :]

In [10]:
# (, "NIR/Red", "VH/VV") (, "Contrast Map", "Dissimilarity Map", "Homogenity Map", "Entropy Map", "Correlation Map", "Mean Map", "Variance Map")
def visualize_indices(image_id, folder_path):
    normalized_img = load_enhanced_normalized_images(image_id, folder_path)
    
    # Original bands count + new indices count
    total_bands = normalized_img.shape[2]
    indices_names = ["Original Bands"] * 12 + ["NDVI", "NDWI", "SAVI"] + ["Slope", "Aspect", "Curvature"]
    
    # Create an appropriate grid size
    rows = (total_bands) // 3
    
    fig, axes = plt.subplots(rows, 4, figsize=(20, 5*rows))
    fig.suptitle(f"Image ID: {image_id} - Original Bands and Spectral Indices", fontsize=16)
    
    # Flatten axes for easier indexing
    axes = axes.flatten()
    
    # Plot each band/index
    for i in range(total_bands):
        if i < len(axes):  # Make sure we don't exceed the number of subplots
            if i == 0:
                cmap = 'Reds'
                axes[i].imshow(normalized_img[:, :, i], cmap=cmap)
                axes[i].set_title(f"Band {i+1}: {band_descriptions[i]}")
            elif i == 1:
                cmap = 'Greens'
                axes[i].imshow(normalized_img[:, :, i], cmap=cmap)
                axes[i].set_title(f"Band {i+1}: {band_descriptions[i]}")
            elif i == 2:
                cmap = 'Blues'
                axes[i].imshow(normalized_img[:, :, i], cmap=cmap)
                axes[i].set_title(f"Band {i+1}: {band_descriptions[i]}")
            elif i < 12:  # Original bands in grayscale
                axes[i].imshow(normalized_img[:, :, i], cmap='viridis')
                axes[i].set_title(f"Band {i+1}: {band_descriptions[i]}")
            else:  # Indices with special colormaps
                if indices_names[i] == "NDVI":
                    cmap = 'RdYlGn'  # Red-Yellow-Green for vegetation
                elif indices_names[i] == "NDWI":
                    cmap = 'Blues'   # Blues for water
                else:
                    cmap = 'viridis' # Default colormap
                
                axes[i].imshow(normalized_img[:, :, i], cmap=cmap)
                axes[i].set_title(f"Index: {indices_names[i]}")
            
            axes[i].axis('off')
    
    # Hide any unused subplots
    for i in range(total_bands, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.3, hspace=0.4)
    plt.show()

In [11]:
# example_ids = train['ID'].sample(1).values

# for ids in example_ids:
#     visualize_indices(ids, train_data)

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-07-15 20:01:59.428026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752609719.631494      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752609719.688233      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score ,confusion_matrix, classification_report
from xgboost import XGBClassifier

In [14]:
X = np.array([load_enhanced_normalized_images(image_id, train_data) for image_id in train['ID']])
y = train['label'].values

In [15]:
# Perform a stratified split to maintain class distribution
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [17]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a simple generator for validation (no augmentation)
val_datagen = ImageDataGenerator()

# Define batch size
batch_size = 32

# custom data generator
# train_ds = TrainDataGenerator(
#     X_train, y_train, batch_size=batch_size, shuffle=True
# )

# val_ds = ValidDataGenerator(
#     X_val, y_val, batch_size=batch_size, shuffle=True
# )

# Create generators using .flow()
train_ds = train_datagen.flow(
    X_train, y_train, batch_size=batch_size, seed=42, shuffle=True
)

val_ds = val_datagen.flow(
    X_val, y_val, batch_size=batch_size, seed=42, shuffle=True
)

In [18]:
X_batch, y_batch = train_ds[0]

In [19]:
X_batch.shape, y_batch.shape

((32, 64, 64, 15), (32,))

In [20]:
X_val_batch, y_val_batch = val_ds[0]

In [21]:
X_val_batch.shape, y_val_batch.shape

((32, 64, 64, 15), (32,))

In [22]:
# Precision metric
def precision_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')  # Cast y_true to float32 to match y_pred type
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# Recall metric
def recall_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')  # Cast y_true to float32 to match y_pred type
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# F1 Score metric
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


# Define the Focal Loss function
def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss for binary classification.

    Parameters:
        gamma (float): Focusing parameter; typically set to 2.0.
        alpha (float): Balancing factor; typically set to 0.25.

    Returns:
        Binary Focal Loss function.
    """
    def focal_loss_fixed(y_true, y_pred):
        # Clip predictions to prevent log(0)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)

        # Calculate focal loss
        fl = -alpha * K.pow(1 - p_t, gamma) * K.log(p_t)
        return K.mean(fl)

    return focal_loss_fixed

In [23]:
# Define the CNN model
# model = Sequential([
#     # First convolutional block
#     Input(shape=X_batch.shape[1:]),
#     Conv2D(32, (3, 3), activation='relu', data_format='channels_last'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     # Second convolutional block
#     Conv2D(64, (3, 3), activation='relu'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     # Third convolutional block
#     Conv2D(128, (3, 3), activation='relu'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),

#     # Fourth convolutional block for deeper feature extraction
#     Conv2D(256, (3, 3), activation='relu'),
#     BatchNormalization(),
#     MaxPooling2D((2, 2)),
#     Dropout(0.5),

#     # Fifth convolutional block for even deeper feature extraction
#     # Conv2D(512, (3, 3), activation='relu'),
#     # BatchNormalization(),
#     # MaxPooling2D((2, 2)),
#     # Dropout(0.5),

#     # Flatten and add dense layers
#     Flatten(),
#     # Dense(256, activation='relu'),
#     # BatchNormalization(),
#     # Dropout(0.5),
#     Dense(128, activation='relu'),
#     BatchNormalization(),
#     Dropout(0.5),  # Dropout for regularization
#     Dense(64, activation='relu'),
#     BatchNormalization(),
#     Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
# ])
# Compile the model with Focal Loss and additional metrics
from tensorflow.keras.applications import ResNet50, EfficientNetB7, MobileNetV2, MobileNetV3Large, DenseNet121
from tensorflow.keras import layers, Model, Input

input_shape = X_batch.shape[1:]
inputs = Input(shape=input_shape)

pre_trained= MobileNetV2(include_top=False, classes=2, pooling='avg', input_shape=input_shape, weights=None)

inp_model = pre_trained.input
x=Flatten()(pre_trained.output)
x=Dense(128, activation='relu')(x)
x=Dropout(0.5)(x)
x=Dense(64, activation='relu')(x)
x=Dropout(0.5)(x)
output=Dense(1, activation='sigmoid')(x)
model = Model(inputs=inp_model, outputs=output)

model.compile(
    optimizer='adam',
    loss=focal_loss(gamma=2.0, alpha=0.5),
    metrics=['accuracy', precision_m, recall_m, f1_m]  # Additional metrics
)

# Display the model summary
model.summary()

I0000 00:00:1752609795.822614      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 64, 15)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │          4,320 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,433,729 (9.28 MB)

 Trainable params: 2,399,617 (9.15 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Create a checkpoint callback that saves the best model based on validation loss
checkpoint = ModelCheckpoint(
    "best_model.keras",            # Filepath to save the model
    monitor='val_f1_m',         # Monitor the validation loss
    verbose=1,                  # Verbosity mode; 1 prints messages when a new best is found
    save_best_only=True,        # Save only the model with the best performance
    mode='max'                  # 'min' mode because lower validation loss is better / 'max' for higher f1 score
)

early_stopping = EarlyStopping(
    monitor='val_f1_m', # Metric to monitor
    mode="max",
    patience=15, # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model using the generators with the checkpoint callback
history = model.fit(
    train_ds,       # Train generator
    epochs=200,
    validation_data=val_ds,  # Validation generator
    callbacks=[checkpoint],  # Include the checkpoint callback in training
    class_weight=class_weights_dict
)

Epoch 1/200


I0000 00:00:1752609835.683104      96 service.cc:148] XLA service 0x7f09200142f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752609835.684113      96 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752609839.048368      96 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1752609843.881952      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609844.085411      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609844.280592      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609844.4634

  1/179 ━━━━━━━━━━━━━━━━━━━━ 2:48:44 57s/step - accuracy: 0.5625 - f1_m: 10.6667 - loss: 0.2594 - precision_m: 8.0000 - recall_m: 16.0000

I0000 00:00:1752609855.269971      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 87/179 ━━━━━━━━━━━━━━━━━━━━ 13s 142ms/step - accuracy: 0.7467 - f1_m: 4.3741 - loss: 0.1219 - precision_m: 5.2351 - recall_m: 4.5790

E0000 00:00:1752609873.860796      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609874.065091      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609874.261332      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752609874.446259      96 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 0.7633 - f1_m: 3.5280 - loss: 0.1024 - precision_m: 4.7096 - recall_m: 3.4909
Epoch 1: val_f1_m improved from -inf to 0.00000, saving model to best_model.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 108s 286ms/step - accuracy: 0.7634 - f1_m: 3.5200 - loss: 0.1023 - precision_m: 4.7024 - recall_m: 3.4818 - val_accuracy: 0.8245 - val_f1_m: 0.0000e+00 - val_loss: 0.0730 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.8259 - f1_m: 0.1214 - loss: 0.0590 - precision_m: 0.3476 - recall_m: 0.0744
Epoch 2: val_f1_m did not improve from 0.00000
179/179 ━━━━━━━━━━━━━━━━━━━━ 26s 145ms/step - accuracy: 0.8259 - f1_m: 0.1212 - loss: 0.0590 - precision_m: 0.3469 - recall_m: 0.0742 - val_accuracy: 0.8245 - val_f1_m: 0.0000e+00 - val_loss: 0.0626 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/200
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.8377 - f1_m

In [ ]:
## Plot training and validation accuracy and loss
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.tight_layout()
plt.show()

In [ ]:
# Loading the full test dataset is not probably the best approach

# Load and predict on the test set
test_ids = test['ID'].values
X_test = np.array([load_enhanced_normalized_images(image_id, test_data) for image_id in test_ids])

In [ ]:
# Predict probabilities and classify as 0 or 1
y_test_pred = (model.predict(X_test) > 0.5).astype(int) # the output is a probability that goes from 0 to 1.
                                                        # Here we use 0.5 as the threshold

# Count the number of predictions for each class
unique, counts = np.unique(y_test_pred, return_counts=True)
prediction_counts = dict(zip(unique, counts))
print("Prediction counts:", prediction_counts)

# Prepare submission file
submission_df = pd.DataFrame({
    'ID': test_ids,
    'label': y_test_pred.flatten()  # Flatten to match submission format
})
submission_df.to_csv('200imagenet_mobilev2.csv', index=False)
print("Sample submission file created as 'all-pca.csv'.")